<a href="https://colab.research.google.com/github/ajayostawal/NLP-distaster-prediction/blob/main/NLP_distaster_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import string
import nltk

In [5]:
train=pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [6]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
test.head()


,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
print(train.shape)
print(test.shape)


(7613, 5)
(3263, 4)


In [9]:
comb=train.append(test,ignore_index=True,sort=True)


In [10]:
comb.shape


(10876, 5)

In [11]:
comb.head


<bound method NDFrame.head of           id keyword  ... target                                               text
0          1     NaN  ...    1.0  Our Deeds are the Reason of this #earthquake M...
1          4     NaN  ...    1.0             Forest fire near La Ronge Sask. Canada
2          5     NaN  ...    1.0  All residents asked to 'shelter in place' are ...
3          6     NaN  ...    1.0  13,000 people receive #wildfires evacuation or...
4          7     NaN  ...    1.0  Just got sent this photo from Ruby #Alaska as ...
...      ...     ...  ...    ...                                                ...
10871  10861     NaN  ...    NaN  EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
10872  10865     NaN  ...    NaN  Storm in RI worse than last hurricane. My city...
10873  10868     NaN  ...    NaN  Green Line derailment in Chicago http://t.co/U...
10874  10874     NaN  ...    NaN  MEG issues Hazardous Weather Outlook (HWO) htt...
10875  10875     NaN  ...    NaN  #CityofCalga

In [13]:
comb.drop('keyword',inplace=True,axis=1)


In [14]:
comb

,id,location,target,text
0,1,NaN,1.0,Our Deeds are the Reason of this #earthquake M...
1,4,NaN,1.0,Forest fire near La Ronge Sask. Canada
2,5,NaN,1.0,All residents asked to 'shelter in place' are ...
3,6,NaN,1.0,"13,000 people receive #wildfires evacuation or..."
4,7,NaN,1.0,Just got sent this photo from Ruby #Alaska as ...
...,...,...,...,...
10871,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
10872,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
10873,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
10874,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [15]:
comb.text=comb.text.apply(lambda x : x.lower())


In [16]:
#Removing puntuation,numbers,special characters
import re
def remove_pun(data):
    pattern='[^A-Za-z0-9\s]'
    data=re.sub(pattern,'',data)
    return data

comb.text=comb.text.apply(remove_pun)

In [17]:
comb.text


0        our deeds are the reason of this earthquake ma...
1                    forest fire near la ronge sask canada
2        all residents asked to shelter in place are be...
3        13000 people receive wildfires evacuation orde...
4        just got sent this photo from ruby alaska as s...
                               ...                        
10871    earthquake safety los angeles  safety fastener...
10872    storm in ri worse than last hurricane my citya...
10873    green line derailment in chicago httptcoutbxlc...
10874    meg issues hazardous weather outlook hwo httpt...
10875    cityofcalgary has activated its municipal emer...
Name: text, Length: 10876, dtype: object

In [18]:
nltk.download('punkt')
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk import tokenize
comb['text']=comb.text.apply(word_tokenize)
comb['text'][0]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['our',
 'deeds',
 'are',
 'the',
 'reason',
 'of',
 'this',
 'earthquake',
 'may',
 'allah',
 'forgive',
 'us',
 'all']

In [19]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer=WordNetLemmatizer()
comb['text']=comb.text.apply(lambda x:[lemmatizer.lemmatize(word) for word in x])
comb.text

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


0        [our, deed, are, the, reason, of, this, earthq...
1            [forest, fire, near, la, ronge, sask, canada]
2        [all, resident, asked, to, shelter, in, place,...
3        [13000, people, receive, wildfire, evacuation,...
4        [just, got, sent, this, photo, from, ruby, ala...
                               ...                        
10871    [earthquake, safety, los, angeles, safety, fas...
10872    [storm, in, ri, worse, than, last, hurricane, ...
10873    [green, line, derailment, in, chicago, httptco...
10874    [meg, issue, hazardous, weather, outlook, hwo,...
10875    [cityofcalgary, ha, activated, it, municipal, ...
Name: text, Length: 10876, dtype: object

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
combine_tfidf = tfidf.fit_transform(comb)

In [21]:
comb


,id,location,target,text
0,1,NaN,1.0,"[our, deed, are, the, reason, of, this, earthq..."
1,4,NaN,1.0,"[forest, fire, near, la, ronge, sask, canada]"
2,5,NaN,1.0,"[all, resident, asked, to, shelter, in, place,..."
3,6,NaN,1.0,"[13000, people, receive, wildfire, evacuation,..."
4,7,NaN,1.0,"[just, got, sent, this, photo, from, ruby, ala..."
...,...,...,...,...
10871,10861,NaN,NaN,"[earthquake, safety, los, angeles, safety, fas..."
10872,10865,NaN,NaN,"[storm, in, ri, worse, than, last, hurricane, ..."
10873,10868,NaN,NaN,"[green, line, derailment, in, chicago, httptco..."
10874,10874,NaN,NaN,"[meg, issue, hazardous, weather, outlook, hwo,..."


In [22]:
text_data= comb['text']

In [25]:
for i in range(len(text_data)):
    text_data[i] = ' '.join(text_data[i])
comb['text']=text_data


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [26]:
comb


,id,location,target,text
0,1,NaN,1.0,our deed are the reason of this earthquake may...
1,4,NaN,1.0,forest fire near la ronge sask canada
2,5,NaN,1.0,all resident asked to shelter in place are bei...
3,6,NaN,1.0,13000 people receive wildfire evacuation order...
4,7,NaN,1.0,just got sent this photo from ruby alaska a sm...
...,...,...,...,...
10871,10861,NaN,NaN,earthquake safety los angeles safety fastener ...
10872,10865,NaN,NaN,storm in ri worse than last hurricane my citya...
10873,10868,NaN,NaN,green line derailment in chicago httptcoutbxlc...
10874,10874,NaN,NaN,meg issue hazardous weather outlook hwo httptc...


In [27]:
test = comb.iloc[7613:,:]


In [28]:
train


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [29]:
test


,id,location,target,text
7613,0,NaN,NaN,just happened a terrible car crash
7614,2,NaN,NaN,heard about earthquake is different city stay ...
7615,3,NaN,NaN,there is a forest fire at spot pond goose are ...
7616,9,NaN,NaN,apocalypse lighting spokane wildfire
7617,11,NaN,NaN,typhoon soudelor kill 28 in china and taiwan
...,...,...,...,...
10871,10861,NaN,NaN,earthquake safety los angeles safety fastener ...
10872,10865,NaN,NaN,storm in ri worse than last hurricane my citya...
10873,10868,NaN,NaN,green line derailment in chicago httptcoutbxlc...
10874,10874,NaN,NaN,meg issue hazardous weather outlook hwo httptc...


In [30]:
test.drop('target',inplace=True,axis=1)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [31]:
test


,id,location,text
7613,0,NaN,just happened a terrible car crash
7614,2,NaN,heard about earthquake is different city stay ...
7615,3,NaN,there is a forest fire at spot pond goose are ...
7616,9,NaN,apocalypse lighting spokane wildfire
7617,11,NaN,typhoon soudelor kill 28 in china and taiwan
...,...,...,...
10871,10861,NaN,earthquake safety los angeles safety fastener ...
10872,10865,NaN,storm in ri worse than last hurricane my citya...
10873,10868,NaN,green line derailment in chicago httptcoutbxlc...
10874,10874,NaN,meg issue hazardous weather outlook hwo httptc...


In [32]:
from sklearn.model_selection import train_test_split
X=train['text']
Y=train['target']

In [33]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.33, random_state=42)


In [34]:
X_train.shape


(5100,)

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
print(X_train_tfidf)

  (0, 2227)	0.4636808849932921
  (0, 3381)	0.08644728447703176
  (0, 7085)	0.09165313511574459
  (0, 12440)	0.348483475729627
  (0, 14224)	0.11198542538782044
  (0, 1639)	0.32531880293496634
  (0, 11388)	0.4636808849932921
  (0, 11229)	0.4636808849932921
  (0, 11004)	0.3151394748465865
  (1, 5053)	0.9455461487945591
  (1, 14224)	0.3254880650650302
  (2, 7704)	0.5213182855832732
  (2, 3792)	0.5213182855832732
  (2, 5243)	0.4474733124599523
  (2, 3819)	0.3748437573746797
  (2, 9089)	0.30927224504745054
  (2, 3381)	0.09719302993813872
  (2, 7085)	0.10304598876782095
  (3, 11493)	0.3344280256731588
  (3, 14612)	0.2488458437022584
  (3, 9659)	0.23277691952494414
  (3, 7395)	0.09619643803847037
  (3, 5845)	0.23939741378273105
  (3, 2082)	0.15805239219601624
  (3, 2876)	0.17305631599707802
  :	:
  (5098, 1423)	0.16716183137109128
  (5098, 11659)	0.24183158195335785
  (5098, 10647)	0.17432939561816427
  (5098, 1728)	0.13781607243845506
  (5098, 7395)	0.09106223925580294
  (5098, 7645)	0.119601

In [36]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train_tfidf, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [37]:
clf=LinearSVC()

In [38]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [39]:
text_clf.fit(X_train, y_train)


Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
               

In [40]:
predictions = text_clf.predict(X_val)


In [41]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_val, predictions))

[[1204  242]
 [ 276  791]]


In [42]:
print(classification_report(y_val, predictions))


              precision    recall  f1-score   support

           0       0.81      0.83      0.82      1446
           1       0.77      0.74      0.75      1067

    accuracy                           0.79      2513
   macro avg       0.79      0.79      0.79      2513
weighted avg       0.79      0.79      0.79      2513



In [43]:
from sklearn import metrics
metrics.accuracy_score(y_val, predictions)

0.7938718662952646

In [44]:
target = text_clf.predict(test['text'])


In [45]:
submission = pd.DataFrame()
submission['id'] = test['id']
submission['target'] = target

In [46]:
submission


,id,target
7613,0,0
7614,2,1
7615,3,1
7616,9,1
7617,11,1
...,...,...
10871,10861,1
10872,10865,0
10873,10868,1
10874,10874,1


In [47]:
submission.set_index('id')


,target
id,
0,0
2,1
3,1
9,1
11,1
...,...
10861,1
10865,0
10868,1


In [48]:
submission.to_csv('./submission.csv')


In [49]:
text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])


In [50]:
text_clf.fit(X_train, y_train)


Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
               

In [51]:
predictions = text_clf.predict(X_val)


In [52]:
from sklearn.metrics import classification_report


In [53]:
print(classification_report(y_val, predictions))


              precision    recall  f1-score   support

           0       0.81      0.83      0.82      1446
           1       0.77      0.74      0.75      1067

    accuracy                           0.79      2513
   macro avg       0.79      0.79      0.79      2513
weighted avg       0.79      0.79      0.79      2513



In [54]:
from sklearn.metrics import accuracy_score


In [56]:
text_clf.fit(X_train, y_train)
predictions = text_clf.predict(X_val)
accuracy = accuracy_score(y_val, predictions)

In [57]:
print(accuracy)


0.7938718662952646


In [58]:
from sklearn.linear_model import LogisticRegression
text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LogisticRegression())])
text_clf.fit(X_train, y_train)
predictions = text_clf.predict(X_val)
accuracy = accuracy_score(y_val, predictions)

In [59]:
print(accuracy)


0.8058097890966972


In [60]:
target = text_clf.predict(test['text'])

submission = pd.DataFrame()
submission['id'] = test['id']
submission['target'] = target
submission.set_index('id')
submission.to_csv('./submission1.csv')